# Imports

In [137]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
# Import all my functions into main name space
from functions.import_all_functions import *
import_all_functions()

import_to_df imported
get_kaggle_data imported


In [139]:
import pandas as pd 

In [140]:
files = get_kaggle_data(competition="titanic")

files downloaded for titanic competition
   test.csv
   train.csv
   gender_submission.csv
located in /competition_data/


In [143]:
input_data = import_to_df(files)

NameError: name 'import_to_df' is not defined

In [125]:
files

['test.csv', 'train.csv', 'gender_submission.csv']